In [5]:
using Distributions
using Seaborn
import StatsBase: wsample
import Clustering: kmeans

In [8]:
include("src/auxiliary.jl")
include("src/chmm.jl")

simulate_model (generic function with 1 method)

In [9]:
K = 3
KK = K^2
D = 2
T = 500
model = rand_chmm(K, D)

N_traj = 3
Zs = Vector{Array{Int}}(N_traj)
X1s = Vector{Array{Float64}}(N_traj)
X2s = similar(X1s)

for n in 1:N_traj
    (Z, X1, X2) = simulate_model(model, T)
    Zs[n] = Z
    X1s[n] = X1
    X2s[n] = X2
end

## EM

In [10]:
# initial EM estiamtes 
R = kmeans([X1 X2], K, maxiter=100, display=:none)
ms = [R.centers[:, i] for i in 1:K]
Ss = [eye(D) for _ in 1:K]

# initial distribition over K × K
p0 = normalize(rand(K), 1)
p0 = (p0 * p0')[:]
log_p0 = log.(p0)

P = rand(K, KK)
P ./= sum(P, 1)
# make it K² × K²
# T[:, i] is symmetric if reshaped to be K×K, ∀i
P = repeat(P, inner=(K, 1)) .* repeat(P, outer=(K, 1))
log_P = log.(P)
;

In [11]:
log_b = zeros(KK, T)
log_α = similar(log_b)
log_β = similar(log_b)

α = similar(log_b)
β = similar(log_b)
γ = similar(log_b)
;

In [12]:
# data across for multiple trajectories
temp_P = zeros(P)
temp_p0 = zeros(p0)
pseudo_counts_K = zeros(K)
pseudo_counts_KK = zeros(KK)

temp_ms = deepcopy(ms)
temp_Ss = deepcopy(Ss)
;

In [ ]:
for i in 1:N_traj
    X1 = X1s[i]
    X2 = X2s[i]

    fill!(temp_P, 0)
    fill!(temp_p0, 0)
    fill!(pseudo_counts_K, 0)
    fill!(pseudo_counts_KK, 0)
    for k in 1:K
        fill!(temp_ms[k], 0)
        fill!(temp_Ss[k], 0)
    end

    for t in 1:T
        for q in 1:K^2
            (i, j) = ind2sub((K, K), q)
            log_b[q, t] = logpdf(MvNormal(ms[i], Ss[i]), X1[:, t]) +
                    logpdf(MvNormal(ms[j], Ss[j]), X2[:, t])
        end
    end

    # forward
    # temp is [ log_a[i, t-1] + log_A[j, i] ]ᵢ
    temp = zeros(KK)
    log_α[:, 1] = log_p0 .+ log_b[:, 1]

    for t in 2:T
        for j in 1:KK
            temp[:] = log_α[:, t-1]
            temp .+= log_P[j, :]
            log_α[j, t] = logsumexp(temp) + log_b[j, t]
        end
    end

    # backward pass
    # temp is [ log_A[i, j] + log_β[j, t+1] + log_b[j, t+1] ]ⱼ
    log_β[:, T] = 0

    for t in (T-1):-1:1
        for i in 1:KK
            temp[:] = log_b[:, t+1]
            temp .+= log_P[:, i]
            temp .+= log_β[:, t+1]
            log_β[i, t] = logsumexp(temp)
        end
    end

    # γ
    for t in 1:T
        γ[:, t] = log_α[:, t]
        γ[:, t] .+= log_β[:, t]
        γ[:, t] .-= logsumexp(γ[:, t])
    end
    map!(exp, γ, γ)

    pseudo_counts_KK .+= squeeze(sum(γ, 2), 2)
    pseudo_counts_K .+= avg_sum(pseudo_counts_KK)

    #
    # parameter update
    #

    # ξ / P
    temp2 = similar(temp_P)
    for t in 1:(T-1)
        temp2[:] = log_P
        temp2 .+= log_α[:, t]'
        temp2 .+= log_b[:, t+1]
        temp2 .+= log_β[:, t+1]
        temp2 .-= logsumexp(temp2)
        temp_P .+= exp.(temp2)
    end

    # π₀
    temp_p0 .+= γ[:, 1]

    # μ & Σ
    for t in 1:T
        p = reshape(γ[:, t], (K, K))
        o1 = X1[:, t]
        o2 = X2[:, t]

        for k in 1:K
            pz1 = sum(p[k, :])
            pz2 = sum(p[:, k])

            temp_ms[k] .+= pz1 * o1
            temp_Ss[k] .+= pz1 * o1 * o1'

            temp_ms[k] .+= pz2 * o2
            temp_Ss[k] .+= pz2 * o2 * o2'
        end
    end
end

In [13]:
### After All Trajectories

temp_P ./= sum(temp_P, 1)
for i in 1:KK
    t = avg_sum(temp_P[:, i])
    P[:, i] = repeat(t, inner=(K, 1)) .* repeat(t, outer=(K, 1))
end
map!(log, log_P, P)

t = avg_sum(temp_p0)
p0[:] = repeat(t, inner=(K, 1)) .* repeat(t, outer=(K, 1))
p0 ./= sum(p0)
map!(log, log_p0, p0)

for k in 1:K
    ms[k][:] = temp_ms[k] ./ (pseudo_counts_K[k] * 2)
    Ss[k][:] = temp_Ss[k] ./ (pseudo_counts_K[k] * 2)
    Ss[k] .-= ms[k] * ms[k]'
end

In [14]:
Z_obs = similar(Z)
for t in 1:T
    Z_obs[:, t] = [ind2sub((K, K), indmax(γ[:, t]))...]
end
sum(Z_obs .!= Z)

651